<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/tnc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from kwta import Sparsify1D_kactive 
from synthetic_data import create_synthetic_data
from torch import nn
from tcn import TemporalConvNet
import torch.nn.functional as F

import torch
import torch.optim as optim

import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    #symbol_sequence= ['circle-open', 'circle', 'circle-open-dot', 'square']
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"],
                                    mode='markers',
                                    marker_size=5,
                                    marker_symbol=all_history[i]["marker_symbol"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()

In [4]:
synth_data = create_synthetic_data(size = 5000)


beat patter dictionary 
[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0]
[0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2]
[0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0] 

[1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0]
[1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 2, 0, 2]
[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
[1, 2,

In [14]:
from my_tests import autoencoder

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)

model = autoencoder(input_size=1, output_size=400, num_channels=[10,25], kernel_size=10, dropout=0.2, wta_k = 5).to(device)
print("first conv kernels size: ", model.get_kernels().cpu().size())

inputs = torch.tensor(synth_data[:,None,:]).float().to(device)
print("Input size: ", inputs.size())
out = model(inputs)
print("Output size: ", out.size(), "\n")


loss_fn = torch.nn.L1Loss().to(device)
optimizer = optim.SGD(model.parameters(), lr=.05, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented
epochs = 3000
history = {"loss": []}
for i in range(epochs):
  optimizer.zero_grad()
  output = model(inputs)

  #decaying WTA
  if i % 500 == 0 and i != 0:
      model.wta.k = max(1, model.wta.k - 1)
      print("model.wta.k: ", model.wta.k)

  loss = loss_fn(output, inputs)
  loss.backward()
  optimizer.step()
  history["loss"].append(float(loss))
  if i % 1 == 0:
      print("Epoch : {} \t Loss : {} \t ".format(i, round(float(loss),7)))
      # print("\nneg encoder ", float((model.encoder.weight.ravel() < 0).sum(dim=0)))






Using device:  cuda
first conv kernels size:  torch.Size([10, 40])
Input size:  torch.Size([5000, 1, 400])
Output size:  torch.Size([5000, 1, 400]) 

Epoch : 0 	 Loss : 5.1847796 	 
Epoch : 1 	 Loss : 0.9741952 	 
Epoch : 2 	 Loss : 0.9500008 	 
Epoch : 3 	 Loss : 0.9499483 	 
Epoch : 4 	 Loss : 0.9498755 	 
Epoch : 5 	 Loss : 0.9498107 	 
Epoch : 6 	 Loss : 0.9497115 	 
Epoch : 7 	 Loss : 0.9495358 	 
Epoch : 8 	 Loss : 0.9491537 	 
Epoch : 9 	 Loss : 0.9481419 	 
Epoch : 10 	 Loss : 0.9453079 	 
Epoch : 11 	 Loss : 0.9414222 	 
Epoch : 12 	 Loss : 0.9374432 	 
Epoch : 13 	 Loss : 0.9315814 	 
Epoch : 14 	 Loss : 0.9238037 	 
Epoch : 15 	 Loss : 0.9128109 	 
Epoch : 16 	 Loss : 0.9002464 	 
Epoch : 17 	 Loss : 0.8816946 	 
Epoch : 18 	 Loss : 0.8878216 	 
Epoch : 19 	 Loss : 0.9304802 	 
Epoch : 20 	 Loss : 0.9165993 	 
Epoch : 21 	 Loss : 0.9001744 	 
Epoch : 22 	 Loss : 0.8670495 	 
Epoch : 23 	 Loss : 0.8721761 	 
Epoch : 24 	 Loss : 0.8711661 	 
Epoch : 25 	 Loss : 0.8281103 	 
Ep

In [15]:

recunstruct = model(torch.tensor(inputs[1:2,:,:]).float())
# print(recunstruct.detach().numpy()[0,0,:])

#perfect plot
original_plot = {"legend": "original", 
                 "x": list(range(0,400)), 
                 "y": synth_data[0,:],
                 "marker_symbol": 'line-ne-open'}
                  
recunstruct_plot = {"legend": "reconstruct", 
             "x": list(range(0,400)), 
             "y": recunstruct.cpu().detach().numpy()[0,0,:],
             "marker_symbol": 'star'}      

plot([recunstruct_plot, original_plot], "Signal Comparison")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [36]:
print("first conv kernels size: ", model.get_kernels().cpu().size())
kernel1 = model.get_kernels().cpu().numpy()[3].tolist()


# kernels6_plot  = {"legend": "original", 
#                  "x": list(range(0,40)), 
#                  "y": kernel6,
#                  "marker_symbol": 'triangle-up-open'}

#perfect plot
kernels1_plot  = {"legend": "original", 
                 "x": list(range(0,40)), 
                 "y": kernel1,
                 "marker_symbol": 'star'}

plot([kernels1_plot], "kernels_plot")

first conv kernels size:  torch.Size([10, 40])


In [46]:
#
kernel = 7

model(torch.tensor(inputs[0:1,:,:]))
feature = model.code.float().detach().cpu().numpy() #all 
print(feature)
feature = model.code.float().detach().cpu().numpy()[0][kernel] #only 3rd 
print(feature)
# feature plot


feature_plot  = {"legend": "feature", 
                 "x": list(range(0,40)), 
                 "y": feature,
                 "marker_symbol": 'star'}



plot([feature_plot], "feature_plot")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



[[[ 0.        0.        0.        0.        0.        0.        0.
    0.        0.        0.      ]
  [ 0.        0.        0.        0.        0.        0.        0.
   15.554894 18.770098  0.      ]
  [ 0.        0.        0.        0.        0.        0.        0.
    0.        0.        0.      ]
  [ 7.929702  8.56782   0.        0.        0.        0.        0.
    0.        0.        0.      ]
  [ 0.        0.        0.        0.        0.       13.614923 15.832678
    0.        0.        0.      ]
  [ 0.        0.        0.        0.       14.746401  0.        0.
    0.        0.        0.      ]
  [ 0.        0.        0.        0.        0.        0.        0.
    0.        0.        0.      ]
  [ 0.        0.       11.772891  0.        0.        0.        0.
    0.        0.       13.769237]
  [ 0.        0.        0.       15.049073  0.        0.        0.
    0.        0.        0.      ]
  [ 0.        0.        0.        0.        0.        0.        0.
    0.        0.  

In [47]:
kernel3 = (model.get_kernels().cpu().numpy()*15)[kernel].tolist()
print(kernel3)


section = 9

kernels3_plot  = {"legend": "kernels2_plot", 
                 "x": list(range(0,40)), 
                 "y": kernel3,
                 "marker_symbol": 'triangle-up-open'}

section2_plot  = {"legend": "section2_plot", 
                 "x": list(range(0,40)), 
                 "y": inputs[0:1,:,:].cpu().numpy().ravel()[section*40:(section+1)*40],
                 "marker_symbol": 'circle'}


print(inputs[0:1,:,:].cpu().numpy().ravel()[section*40:(section+1)*40])

plot([kernels3_plot,section2_plot], "kernel and features")

[1.0853705406188965, 2.290388584136963, 1.0630755424499512, 0.020259546115994453, 1.0853705406188965, 2.2903900146484375, 1.0630757808685303, 0.020409470424056053, 1.0853700637817383, 2.2903943061828613, 1.0630731582641602, 0.012618944980204105, 1.0853753089904785, 2.2903900146484375, 1.063075065612793, 0.012738708406686783, 1.0853716135025024, 2.2903926372528076, 1.0630720853805542, 0.012065054848790169, 1.0853705406188965, 2.2903900146484375, 1.063072919845581, 0.01222244556993246, 1.0853711366653442, 2.2903926372528076, 1.063071846961975, 0.01234673522412777, 1.085369348526001, 2.290388584136963, 1.0630756616592407, 0.012748431414365768, 1.0853716135025024, 2.290388584136963, 1.0630751848220825, 0.016649819910526276, 1.085375428199768, 2.290386915206909, 1.0630697011947632, 0.011693411506712437]
[1. 2. 1. 0. 1. 2. 1. 0. 1. 2. 1. 0. 1. 2. 1. 0. 1. 2. 1. 0. 1. 2. 1. 0.
 1. 2. 1. 0. 1. 2. 1. 0. 1. 2. 1. 0. 1. 2. 1. 0.]
